In [1]:
import sys
import os
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from tensorflow.keras.utils import to_categorical


df=pd.read_csv('fer2013.csv')
print(df.info)

<bound method DataFrame.info of        emotion                                             pixels        Usage
0            0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...     Training
1            0  151 150 147 155 148 133 111 140 170 174 182 15...     Training
2            2  231 212 156 164 174 138 161 173 182 200 106 38...     Training
3            4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...     Training
4            6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...     Training
...        ...                                                ...          ...
35882        6  50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...  PrivateTest
35883        3  178 174 172 173 181 188 191 194 196 199 200 20...  PrivateTest
35884        0  17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...  PrivateTest
35885        3  30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...  PrivateTest
35886        2  19 13 14 12 13 16 21 33 50 57 71 84 97 108 122...  PrivateTest

[35887 rows x 3 col

In [2]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install numpy


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
print(df["Usage"].value_counts)

<bound method IndexOpsMixin.value_counts of 0           Training
1           Training
2           Training
3           Training
4           Training
            ...     
35882    PrivateTest
35883    PrivateTest
35884    PrivateTest
35885    PrivateTest
35886    PrivateTest
Name: Usage, Length: 35887, dtype: object>


In [9]:
print(df.head(10))

   emotion                                             pixels     Usage
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...  Training
1        0  151 150 147 155 148 133 111 140 170 174 182 15...  Training
2        2  231 212 156 164 174 138 161 173 182 200 106 38...  Training
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...  Training
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...  Training
5        2  55 55 55 55 55 54 60 68 54 85 151 163 170 179 ...  Training
6        4  20 17 19 21 25 38 42 42 46 54 56 62 63 66 82 1...  Training
7        3  77 78 79 79 78 75 60 55 47 48 58 73 77 79 57 5...  Training
8        3  85 84 90 121 101 102 133 153 153 169 177 189 1...  Training
9        2  255 254 255 254 254 179 122 107 95 124 149 150...  Training


In [11]:
X_train,train_y,X_test,test_y=[],[],[],[]

In [13]:
for index,row in df.iterrows():
    val=row["pixels"].split(" ")
    if 'Training' in row["Usage"]:
            X_train.append(np.array(val,'float32'))
            train_y.append(row['emotion'])
    elif 'PublicTest' in row["Usage"]:
            X_test.append(np.array(val,'float32'))
            test_y.append(row['emotion'])
    
print(f"X train sample data: {X_train[0:2]}")
print(f"y train sample data: {train_y[0:2]}")
print(f"X test sample data: {X_test[0:2]}")
print(f"y test sample data: {test_y[0:2]}")


X train sample data: [array([ 70.,  80.,  82., ..., 106., 109.,  82.], dtype=float32), array([151., 150., 147., ..., 193., 183., 184.], dtype=float32)]
y train sample data: [0, 0]
X test sample data: [array([254., 254., 254., ...,  42., 129., 180.], dtype=float32), array([156., 184., 198., ..., 172., 167., 161.], dtype=float32)]
y test sample data: [0, 1]


In [14]:
X_train=np.array(X_train,'float32')
train_y=np.array(train_y,'float32')
X_test=np.array(X_test,'float32')
test_y=np.array(test_y,'float32')
from keras.utils import to_categorical



#Normalizing data between 0 and 1
X_train-=np.mean(X_train,axis=0)
X_train/=np.std(X_train,axis=0)

X_test-=np.mean(X_test,axis=0)
X_test/=np.std(X_test,axis=0)

num_features=64
num_labels=7
batch_size=64
epochs=30
width,height=48,48
X_train=X_train.reshape(X_train.shape[0],width,height,1)
X_test=X_test.reshape(X_test.shape[0],width,height,1)

# Convert labels to one-hot encoded format
train_y = to_categorical(train_y, num_classes=num_labels)
test_y = to_categorical(test_y, num_classes=num_labels)

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy

# Model definition
model = Sequential()

# First convolutional block
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Dropout(0.5))

# Second convolutional block
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Dropout(0.5))

# Third convolutional block
model.add(Conv2D(2 * num_features, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(2 * num_features, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

# Flattening and fully connected layers
model.add(Flatten())
model.add(Dense(2 * 2 * 2 * 2 * num_features, activation='relu'))  # Ensure this size matches your architecture
model.add(Dropout(0.2))
model.add(Dense(num_labels, activation='softmax'))

# Compile the model
model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])

# Training the model
model.fit(
    X_train,
    train_y,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(X_test, test_y),
    shuffle=True
)

# Save the model structure and weights
fer_json = model.to_json()
with open("fer.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer.weights.h5")


C:\Users\BhumikaMallapur\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 154s 335ms/step - accuracy: 0.2612 - loss: 1.7922 - val_accuracy: 0.3937 - val_loss: 1.5907
Epoch 2/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 149s 333ms/step - accuracy: 0.3949 - loss: 1.5386 - val_accuracy: 0.4817 - val_loss: 1.3504
Epoch 3/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 146s 325ms/step - accuracy: 0.4693 - loss: 1.3748 - val_accuracy: 0.5110 - val_loss: 1.2749
Epoch 4/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 146s 325ms/step - accuracy: 0.5005 - loss: 1.2987 - val_accuracy: 0.5199 - val_loss: 1.2361
Epoch 5/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 136s 303ms/step - accuracy: 0.5248 - loss: 1.2488 - val_accuracy: 0.5386 - val_loss: 1.2051
Epoch 6/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 130s 289ms/step - accuracy: 0.5397 - loss: 1.2044 - val_accuracy: 0.5383 - val_loss: 1.1890
Epoch 7/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 139s 310ms/step - accuracy: 0.5545 - loss: 1.1692 - val_accuracy: 0.5475 - val_loss: 1.1880
Epoch 8/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 143s 319ms/step - accuracy: 0.5646 -

In [17]:
num_features=64
num_labels=7
batch_size=64
epochs=30
width,height=48,48
X_train=X_train.reshape(X_train.shape[0],width,height,1)
X_test=X_test.reshape(X_test.shape[0],width,height,1)


In [19]:
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0


In [21]:
print(np.any(np.isnan(X_train)), np.any(np.isinf(X_train)))


False False


In [23]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy

# Model definition
model = Sequential()

# First convolutional block
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Dropout(0.5))

# Second convolutional block
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Dropout(0.5))

# Third convolutional block
model.add(Conv2D(2 * num_features, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(2 * num_features, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

# Flattening and fully connected layers
model.add(Flatten())
model.add(Dense(2 * 2 * 2 * 2 * num_features, activation='relu'))  # Ensure this size matches your architecture
model.add(Dropout(0.2))
model.add(Dense(num_labels, activation='softmax'))

# Compile the model
model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])

# Training the model
model.fit(
    X_train,
    train_y,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(X_test, test_y),
    shuffle=True
)

# Save the model structure and weights
fer_json = model.to_json()
with open("fer.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer.h5")


C:\Users\BhumikaMallapur\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 211s 461ms/step - accuracy: 0.2533 - loss: 1.8222 - val_accuracy: 0.2494 - val_loss: 1.8142
Epoch 2/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 251s 559ms/step - accuracy: 0.2505 - loss: 1.8124 - val_accuracy: 0.2494 - val_loss: 1.8115
Epoch 3/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 146s 325ms/step - accuracy: 0.2513 - loss: 1.8112 - val_accuracy: 0.2494 - val_loss: 1.8131
Epoch 4/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 146s 325ms/step - accuracy: 0.2517 - loss: 1.8150 - val_accuracy: 0.2494 - val_loss: 1.8117
Epoch 5/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 143s 318ms/step - accuracy: 0.2452 - loss: 1.8149 - val_accuracy: 0.2494 - val_loss: 1.8141
Epoch 6/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 154s 344ms/step - accuracy: 0.2481 - loss: 1.8146 - val_accuracy: 0.2494 - val_loss: 1.8138
Epoch 7/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 157s 350ms/step - accuracy: 0.2536 - loss: 1.8096 - val_accuracy: 0.2494 - val_loss: 1.8110
Epoch 8/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 185s 411ms/step - accuracy: 0.2529 -

ValueError: The filename must end in `.weights.h5`. Received: filepath=fer.h5

In [14]:
fer_json = model.to_json()
with open("fer.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer.weights.h5")